# Setting up and training simple convolutional networks with tensorflow and Keras

In this tutorial, we will learn how to set up simple 3D convolutional network with tensorflow and keras.
As an example, we will set up a network that takes a batch of 3D tensors with 2 channels (e.g. PET and MR) as input and outputs a batch of 3D tensors with 1 channel (denoised and deblurred PET image).
Moreover, we will see how to train a model and how to monitor training. 

The model that we will setup in this tutorial will look like the figure below, except that we won't split and
concatenate the features in the first layer.

![foo bar](https://raw.githubusercontent.com/gschramm/pyapetnet/master/figures/fig_1_apetnet.png)

In [ ]:
# import python modules used in this tutorial

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
from tempfile import NamedTemporaryFile

## Setting up a simple network

Before setting up our first model, we define a short helper function that allows us to visualize models in a matplotlib figure.

In [ ]:
def show_model(model):
  """ function that saves structure of a model into png and shows it with matplotlib
  """
  tmp_file = NamedTemporaryFile(prefix = 'model', suffix = '.png')
  tf.keras.utils.plot_model(model, to_file= tmp_file.name, show_shapes = True, dpi = 192)
  img = mpimg.imread(tmp_file)
  fig, ax = plt.subplots(figsize = (12,12))
  img = plt.imshow(img)
  ax.set_axis_off()

  return fig, ax

Let's setup the network described above. We can setup the whole network with layers that are predefined in keras which makes life easy. Since our desired output (denoised and beblurred PET image) is "close" to first input channel (the noisy and blurry PET image), we add the first input channel to the output. The batch and spatial dimensions of all layers are "None", since all layers preserve those dimensions. This in turn means the model an be applied to all batch sizes and spatial dimensions. 

In [ ]:
# model parameters
nfeat          = 20      # number of featuers for Conv3D layers
kernel_shape   = (3,3,3) # kernel shapes for Conv3D layers
nhidden_layers = 2       # number of hiddenlayers 
batch_norm     = True    # use batch normalization between Conv3D and activation
add_final_relu = True    # add a final ReLU activation at the end to clip negative values

#---------------------------------------------------------------------------------------------------------------------

# setup the input layer for batches of 3D tensors with two channels
inp = tf.keras.layers.Input(shape = (None, None, None, 2), name = 'input_layer')

# add a split layer such that we can add the first channel (PET) to the output
split = tf.keras.layers.Lambda( lambda x: tf.split(x, num_or_size_splits = 2, axis = -1), name = 'split')(inp)

# add all "hidden" layers
x   = inp
for i in range(nhidden_layers):
  x = tf.keras.layers.Conv3D(nfeat, kernel_shape, padding = 'same',
                             kernel_initializer = 'glorot_uniform', name = f'conv3d_{i+1}')(x)
  if batch_norm:
    x = tf.keras.layers.BatchNormalization(name = f'batchnorm_{i+1}')(x)
  x = tf.keras.layers.PReLU(shared_axes=[1,2,3], name = f'prelu_{i+1}')(x)


# add a (1,1,1) Conv layers with 1 feature to reduce along the feature dimension
x = tf.keras.layers.Conv3D(1, (1,1,1), padding='same', name = 'conv_final',
                           kernel_initializer = 'glorot_uniform')(x)

# add first input channel
x = tf.keras.layers.Add(name = 'add')([x] + [split[0]])

# add a final ReLU to clip negative values
if add_final_relu:
  x = tf.keras.layers.ReLU(name = 'final_relu')(x)

model  = tf.keras.Model(inputs = inp, outputs = x)

Let's print a summary of all layers, connections and the number of trainable parameters.

In [ ]:
print(model.summary())

Let's visualize the model using the helper function defined above.

In [ ]:
fig, ax = show_model(model)